In [2]:
# !pip3 install cvxpy
# !pip3 install Mosek

In [1]:
import os
import pathlib
import sys

import pandas as pd
import torch
import torchvision
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
import seaborn as sns
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"device = {device}")
import cp.transformations as cp_t
import cp.graph_transformations as cp_gt
from cp.graph_cp import GraphCP

from scipy.stats import norm

from utils import ModelManager
from utils import standard_l2_norm

# assignments
datasets_folder = "path_to_dataset"
models_direction = "path_to_models"


import cvxpy as convex


device = cuda
Torch Graph Models are running on cuda
v16


In [2]:
import pickle

def save_pkl(obj, path):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
def load_pkl(path):
    with open(path, 'rb') as f:
        return pickle.load(f)


In [3]:
def get_cal_mask(vals_tensor, fraction=0.1):
    perm = torch.randperm(vals_tensor.shape[0])
    mask = torch.zeros((vals_tensor.shape[0]), dtype=bool)
    cutoff_index = int(vals_tensor.shape[0] * fraction)
    mask[perm[:cutoff_index]] = True
    return mask

def singleton_hit(pred_set, y_true):
    return ((pred_set[y_true])[pred_set.sum(axis=1) == 1].sum() / (pred_set).shape[0]).item()

In [5]:
randoms = (torch.rand((1000,)) * (0.73 - 0.72)) + 0.72
sigma = 0.01

def np_upperbound(randoms, SIGMA, radi, alpha=0.05, n_classes=1):
    bon_alpha = alpha / n_classes
    error = 0
    p_upper = torch.minimum(randoms.mean() + error, torch.tensor(1.0).to(randoms.device))
    result = norm.cdf(
        norm.ppf(p_upper.cpu(), scale=SIGMA) + radi,
        scale=SIGMA)
    return torch.tensor(result)
print("NP bound: ", np_upperbound(randoms, pert_radi, sigma))

def dkw_upperbound(randoms, SIGMA, radi, alpha=0.05, num_s=1000, n_classes=1, evasion=True):
    bon_alpha = alpha / n_classes
    error = 0
    s_min = 0
    s_max = 1
    s_seg = torch.linspace(s_min, s_max, num_s + 1)

    empi_cdf = torch.minimum(
        ((randoms.view(-1, 1) > s_seg.to(randoms.device)).sum(dim=0) / randoms.shape[0]) + error,
        torch.tensor([1.0]).to(randoms.device))

    result = (norm.cdf(norm.ppf(empi_cdf.cpu(), scale=SIGMA) + radi, scale=SIGMA) * (1 / (num_s))).sum() + (1/num_s)
    return torch.tensor(result)
print("DKW bound: ", dkw_upperbound(randoms, pert_radi, sigma))


def dkw_lowerbound(randoms, SIGMA, radi, alpha=0.05, num_s=1000, n_classes=1, evasion=True):
    bon_alpha = alpha / n_classes
    error = 0
    s_min = 0
    s_max = 1
    s_seg = torch.linspace(s_min, s_max, num_s + 1)

    empi_cdf = torch.maximum(
        ((randoms.view(-1, 1) > s_seg.to(randoms.device)).sum(dim=0) / randoms.shape[0]) - error,
        torch.tensor([0.0]).to(randoms.device))

    result = (norm.cdf(norm.ppf(empi_cdf.cpu(), scale=SIGMA) - radi, scale=SIGMA) * (1 / (num_s))).sum()
    return torch.tensor(result)


def np_upperbound_tensor(scores_samplings, SIGMA, radi, alpha=0.05, n_classes=1):
    bon_alpha = alpha / n_classes
    # error = np.sqrt(np.log(1 / bon_alpha) / (2 * scores_samplings.shape[-1]))
    error = 0
    p_uppers = torch.minimum(scores_samplings.mean(dim=-1) + error, torch.tensor(1.0).to(scores_samplings.device))
    result = norm.cdf(
        norm.ppf(p_uppers.cpu(), scale=SIGMA) + radi,
        scale=SIGMA)
    return torch.tensor(result).to(scores_samplings.device)

def dkw_upperbound_tensor(scores_sampling, SIGMA, radi, alpha=0.05, num_s=10000, n_classes=1):
    return torch.stack([
        torch.stack([
            dkw_upperbound(scores_sampling[d, c, :], SIGMA=SIGMA, radi=radi, alpha=alpha, num_s=num_s, n_classes=n_classes)
            for c in range(scores_sampling.shape[1])
        ])
        for d in range(scores_sampling.shape[0])
    ]).to(scores_sampling.device)

def dkw_lowerbound_tensor(scores_sampling, SIGMA, radi, alpha=0.05, num_s=10000, n_classes=1):
    return torch.stack([
        torch.stack([
            dkw_lowerbound(scores_sampling[d, c, :], SIGMA=SIGMA, radi=radi, alpha=alpha, num_s=num_s, n_classes=n_classes)
            for c in range(scores_sampling.shape[1])
        ])
        for d in range(scores_sampling.shape[0])
    ]).to(scores_sampling.device)


def np_bounds_tensor(scores_samplings, SIGMA, radi, alpha=0.05, n_classes=1):
    bon_alpha = alpha / n_classes
    error = 0
    p_uppers = torch.minimum(scores_samplings.mean(dim=-1) + error, torch.tensor(1.0).to(scores_samplings.device))
    p_lowers = torch.maximum(scores_samplings.mean(dim=-1) - error, torch.tensor(0.0).to(scores_samplings.device))

    upper_result = norm.cdf(
        norm.ppf(p_uppers.cpu(), scale=SIGMA) + radi,
        scale=SIGMA)
    lower_result = norm.cdf(
        norm.ppf(p_lowers.cpu(), scale=SIGMA) - radi,
        scale=SIGMA)

    return torch.tensor(lower_result).to(scores_samplings.device), torch.tensor(upper_result).to(scores_samplings.device)


In [7]:
def find_worst_quantile_budget(clean_scores, cal_lower, cal_upper, coverage, no_pert=10):
    if isinstance(cal_lower, torch.Tensor):
        cal_lower = cal_lower.cpu().numpy()
    if isinstance(cal_upper, torch.Tensor):
        cal_upper = cal_upper.cpu().numpy()
    if isinstance(clean_scores, torch.Tensor):
        clean_scores = clean_scores.cpu().numpy()
    
    alpha = 1 - coverage  # quantile level
    M = 10  # large positive number (should be as small as possible)

    n = cal_lower.shape[0]

    # Variables
    y = convex.Variable(n)
    q = convex.Variable()
    z = convex.Variable(n, boolean=True)
    w = convex.Variable(n, boolean=True)



    # Constraints
    constraints = [
        cal_lower <= y,
        y <= cal_upper,
        convex.sum(z) >= convex.floor(alpha * n),
        convex.sum(1 - z) >= (1 - alpha) * n,
        y <= q + M * (1 - z),
        y >= q - M * z,

        convex.sum(w) <= no_pert, #number of perturbed points,
        y <= clean_scores + M * w,
        y >= clean_scores - M * w
    ]

    prob = convex.Problem(convex.Maximize(q), constraints).solve(verbose=False, solver='MOSEK')
    quantile = torch.tensor(q.value)
    return quantile

def find_best_quantile_budget(clean_scores, cal_lower, cal_upper, coverage, no_pert=10):
    if isinstance(cal_lower, torch.Tensor):
        cal_lower = cal_lower.cpu().numpy()
    if isinstance(cal_upper, torch.Tensor):
        cal_upper = cal_upper.cpu().numpy()
    if isinstance(clean_scores, torch.Tensor):
        clean_scores = clean_scores.cpu().numpy()
    
    alpha = 1 - coverage  # quantile level
    M = 10  # large positive number (should be as small as possible)

    n = cal_lower.shape[0]

    # Variables
    y = convex.Variable(n)
    q = convex.Variable()
    z = convex.Variable(n, boolean=True)
    w = convex.Variable(n, boolean=True)



    # Constraints
    constraints = [
        cal_lower <= y,
        y <= cal_upper,
        convex.sum(z) >= convex.floor(alpha * n),
        convex.sum(1 - z) >= (1 - alpha) * n,
        y <= q + M * (1 - z),
        y >= q - M * z,

        convex.sum(w) <= no_pert, #number of perturbed points,
        y <= clean_scores + M * w,
        y >= clean_scores - M * w
    ]

    prob = convex.Problem(convex.Minimize(q), constraints).solve(verbose=False, solver='MOSEK')
    quantile = torch.tensor(q.value)
    return quantile


def find_worst_quantile(clean_scores, cal_lower, cal_upper, coverage):
    if isinstance(cal_lower, torch.Tensor):
        cal_lower = cal_lower.cpu().numpy()
    if isinstance(cal_upper, torch.Tensor):
        cal_upper = cal_upper.cpu().numpy()
    if isinstance(clean_scores, torch.Tensor):
        clean_scores = clean_scores.cpu().numpy()
    
    alpha = 1 - coverage  # quantile level
    M = 10  # large positive number (should be as small as possible)

    n = cal_lower.shape[0]

    # Variables
    y = convex.Variable(n)
    q = convex.Variable()
    z = convex.Variable(n, boolean=True)
    w = convex.Variable(n, boolean=True)



    # Constraints
    constraints = [
        cal_lower <= y,
        y <= cal_upper,
        convex.sum(z) >= convex.floor(alpha * n),
        convex.sum(1 - z) >= (1 - alpha) * n,
        y <= q + M * (1 - z),
        y >= q - M * z,
    ]

    prob = convex.Problem(convex.Maximize(q), constraints).solve(verbose=False, solver='MOSEK')
    quantile = torch.tensor(q.value)
    return quantile

def find_best_quantile(clean_scores, cal_lower, cal_upper, coverage):
    if isinstance(cal_lower, torch.Tensor):
        cal_lower = cal_lower.cpu().numpy()
    if isinstance(cal_upper, torch.Tensor):
        cal_upper = cal_upper.cpu().numpy()
    if isinstance(clean_scores, torch.Tensor):
        clean_scores = clean_scores.cpu().numpy()
    
    alpha = 1 - coverage  # quantile level
    M = 10  # large positive number (should be as small as possible)

    n = cal_lower.shape[0]

    # Variables
    y = convex.Variable(n)
    q = convex.Variable()
    z = convex.Variable(n, boolean=True)
    w = convex.Variable(n, boolean=True)



    # Constraints
    constraints = [
        cal_lower <= y,
        y <= cal_upper,
        convex.sum(z) >= convex.floor(alpha * n),
        convex.sum(1 - z) >= (1 - alpha) * n,
        y <= q + M * (1 - z),
        y >= q - M * z,
    ]

    prob = convex.Problem(convex.Minimize(q), constraints).solve(verbose=False, solver='MOSEK')
    quantile = torch.tensor(q.value)
    return quantile



In [8]:
y_pred, logits, y_true = load_pkl(f'path_to_logits')
y_pred = y_pred.to(device)
logits = logits.to(device)
y_true = y_true.to(device)
smoothing_sigma = 0.25
pert_radi = 0.12

y_true_mask = F.one_hot(y_true).bool()

#APS
cp = GraphCP(transformation_sequence=[cp_t.APSTransformation(softmax=True)], coverage_guarantee=0.9)
sc_scores = torch.stack([cp.get_scores_from_logits(logits[:, i, :]) for i in range(logits.shape[1])]).permute(1, 2, 0) + 1
esc_scores = sc_scores.mean(axis=2)
esc_scores.shape

n_classes = 10


In [9]:
np.array([0, 0.25, 0.5, 0.75, 1, 1.5]) * smoothing_sigma

array([0.    , 0.0625, 0.125 , 0.1875, 0.25  , 0.375 ])

In [10]:
coverage = 0.9
coverages = np.array([0.8, 0.85, 0.9, 0.95]) 
pert_radis =  np.array([ 0.25, 0.5, 0.75]) * smoothing_sigma
nb_results = []

for pert_radi in pert_radis:
    np_lower, np_upper = np_bounds_tensor(sc_scores, SIGMA=smoothing_sigma, radi=pert_radi, alpha=0.05, n_classes=n_classes)
    for coverage in coverages:
        for iter_i in tqdm(range(100)):
            cal_mask = get_cal_mask(sc_scores, fraction=0.1)
            cal_lower = np_lower[cal_mask][y_true_mask[cal_mask]]
            cal_upper = np_upper[cal_mask][y_true_mask[cal_mask]]
            cal_scores = esc_scores[cal_mask][y_true_mask[cal_mask]]
            eval_mask = ~cal_mask


            best_q = find_best_quantile(cal_scores, cal_lower, cal_upper, coverage=coverage)

            best_pred_sets = (esc_scores >= best_q)

            nb_results.append({
                "iter_i": iter_i,
                'best_q': best_q.item(),
                'actual_q': cp.calibrate_from_scores(esc_scores[cal_mask], y_true_mask[cal_mask]),
                'best_coverage': cp.coverage(best_pred_sets[eval_mask], y_true_mask[eval_mask]),
                'best_ave_set_size': cp.average_set_size(best_pred_sets[eval_mask]),
                'best_singleton_hits': singleton_hit(best_pred_sets, y_true_mask),
                '$1-\alpha$': coverage,
                'radi': pert_radi

            })

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [05:19<00:00,  3.19s/it]


In [11]:
pd.DataFrame(nb_results).to_csv('./poisoning_np_results.csv')

In [14]:
coverages = np.array([0.8, 0.85, 0.9, 0.95]) 
pert_radis =  np.array([0.25, 0.5, 0.75]) * smoothing_sigma
dnp_results = []
for pert_radi in pert_radis:
    for coverage in coverages:
        dkw_lower = dkw_lowerbound_tensor(sc_scores, SIGMA=smoothing_sigma, radi=pert_radi, alpha=0.05, num_s=10000, n_classes=n_classes)
        dkw_upper = dkw_upperbound_tensor(sc_scores, SIGMA=smoothing_sigma, radi=pert_radi, alpha=0.05, num_s=10000, n_classes=n_classes)
        for iter_i in tqdm(range(100)):
            cal_mask = get_cal_mask(sc_scores, fraction=0.1)
            cal_lower = dkw_lower[cal_mask][y_true_mask[cal_mask]]
            cal_upper = dkw_upper[cal_mask][y_true_mask[cal_mask]]
            cal_scores = esc_scores[cal_mask][y_true_mask[cal_mask]]
            eval_mask = ~cal_mask


            best_q = find_best_quantile(cal_scores, cal_lower, cal_upper, coverage=coverage)

            best_pred_sets = (esc_scores >= best_q)

            dnp_results.append({
                "iter_i": iter_i,
                'best_q': best_q.item(),
                'actual_q': cp.calibrate_from_scores(esc_scores[cal_mask], y_true_mask[cal_mask]),
                'best_coverage': cp.coverage(best_pred_sets[eval_mask], y_true_mask[eval_mask]),
                'best_ave_set_size': cp.average_set_size(best_pred_sets[eval_mask]),
                'best_singleton_hits': singleton_hit(best_pred_sets, y_true_mask),
                '$1-\alpha$': coverage,
                'radi': pert_radi
            })

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:48<00:00,  2.07it/s]


In [ ]:
pd.DataFrame(dnp_results).to_csv('./poisoning_dkw_results.csv')

# constraint optimization (with budget)

In [ ]:
pert_budget_range = [1,2,3]
coverage = 0.85

for iter_i in tqdm(range(100)):
    cal_mask = get_cal_mask(sc_scores, fraction=0.1)
    cal_lower = np_lower[cal_mask][y_true_mask[cal_mask]]
    cal_upper = np_upper[cal_mask][y_true_mask[cal_mask]]
    cal_scores = esc_scores[cal_mask][y_true_mask[cal_mask]]
    eval_mask = ~cal_mask
    
    for pert_budget in pert_budget_range:

        best_q = find_best_quantile_budget(cal_scores, cal_lower, cal_upper, coverage=coverage, no_pert=pert_budget)

        best_pred_sets = (esc_scores >= best_q)
    

        results.append({
            "iter_i": iter_i,
            'pert_budget': pert_budget,
            'best_q': best_q.item(),
            'actual_q': cp.calibrate_from_scores(esc_scores[cal_mask], y_true_mask[cal_mask]),
            'best_coverage': cp.coverage(best_pred_sets[eval_mask], y_true_mask[eval_mask]),
            'best_ave_set_size': cp.average_set_size(best_pred_sets[eval_mask]),
            'best_singleton_hits': singleton_hit(best_pred_sets, y_true_mask),
            'coverage': coverage
        })

In [ ]:
pd.DataFrame(results).to_csv('./poisoning_wbudget.csv')
